# Business Context

## Business Challenge

Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um modelo de negócio do tipo E-commerce.

A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação baixo e escalar a medida que forem conseguindo clientes.

Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e o material para a fabricação de cada peça.

Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes perguntas: 

1. Qual o melhor preço de venda para as calças? 

2. Quantos tipos de calças e suas cores para o produto inicial? 

3. Quais as matérias-prima necessárias para confeccionar as calças?

As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

## Planejamento da Solução (Método SAPE)

### 1.1. Questão 1 (Tipos de Calças e Suas Cores)

#### 1.1.1. Problema do Negócio

- Quantos tipos de calças e suas cores para o produto inicial?

#### 1.1.2. Saída: (Produto Final)

***1. A resposta para a pergunta***
   
   - selecionar os tipos de calças com cores com melhor custo benefícios.
    
***2. Formato da entrega***
    
   - Tabela ou gráfico.
    
***3. Local da entrega***
    
   - App Streamlit.

#### 1.1.3. Processo (Passo a Passo)

***1. Passo a passo para construir o cálculo do custo benefício.***

   - Realizar a subtração do valor de venda menos o valor de produção por tipo-cor.
    
***2. Definir o formato da entrega (Visualização, Tabela, Frase):***

   - Definição de gráfico de barras com o custo benefício de cada tipo-cor.
   - Tabela com as seguintes colunas: id | product_name | type | product_color | cost | price | cost_benefit
   - Definição do schema: colunas e seu tipo
   - Definição da infraestrutura de armazenamento (SQLITE3)
   - Design do ETL (scripts de extração, transformação e carga)
   - Planejamento de agendamento dos scripts (dependências entre os scripts)
   - Fazer as visualizações
   - Entrega do produto final
    
***3. Decidir o local de entrega (PowerBI, Telegram, Email, Streamlit, Intranet)***
    
   - App com Streamlit

### 1.2. Questão 2 (Matérias Primas)

#### 1.1.1. Problema do Negócio

- Quais as matérias-prima necessárias para confeccionar as calças?

#### 1.1.2. Saída: (Produto Final)

***1. A resposta para a pergunta***
   
   - realizar um estudo sobre as matérias primas necessárias na confecção das calças.
    
***2. Formato da entrega***
    
   - Tabela ou gráfico.
    
***3. Local da entrega***
    
   - App Streamlit.

#### 1.1.3. Processo (Passo a Passo)

***1. Passo a passo para construir o cálculo do custo benefício.***

   - Realizar a extração dos dados da fontes selecionadas.
    
***2. Definir o formato da entrega (Visualização, Tabela, Frase):***

   - Tabela com as seguintes colunas: id | product_name | type | product_color | cost | price | cost_benefit
   - Definição do schema: colunas e seu tipo
   - Definição da infraestrutura de armazenamento (SQLITE3)
   - Design do ETL (scripts de extração, transformação e carga)
   - Planejamento de agendamento dos scripts (dependências entre os scripts)
   - Fazer as visualizações
   - Entrega do produto final
    
***3. Decidir o local de entrega (PowerBI, Telegram, Email, Streamlit, Intranet)***
    
   - App com Streamlit

### 1.3. Entrada (Fonte de Dados)

***1. Fonte de dados***

   - Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
   - Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans
   
***2. Ferramentas***

- Python 3.9.4
- Bibliotecas de Webscrapping ( BS4, Selenium )
- PyCharm
- Jupyter Notebook ( Analise e prototipagens )
- Crontjob, Airflow
- Streamlit

# Libraries

In [1]:
# manipulação de dados
import pandas as pd
from datetime import datetime
import numpy as np

# webscrapping
from bs4 import BeautifulSoup
import requests

# regex
import re

# SQL
import sqlite3
from sqlalchemy import create_engine

# 1.0. Step 01 - Collecting Data

## 1.1. Defining Ways

In [2]:
#defining the url
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

#defining the header
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

In [3]:
#defining parser
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
#selecting start class
products = soup.find('ul', class_='products-listing small')

## 1.2. Extracting Data

### 1.2.1. Extract I

In [6]:
# obtendo os dados dos produtos
product_list = products.find_all('article', class_='hm-product-item')
product_list[2].get('data-articlecode')

#product_id
product_id = [p.get('data-articlecode') for p in product_list]

#product_category
product_cat = [p.get('data-category') for p in product_list]

In [7]:
# obtendo os dados dos produtos
product_list = products.find_all('a', class_='link')
product_list[0].get_text()

#product_name
product_name = [p.get_text() for p in product_list]

In [8]:
# price
product_list = products.find_all('span', class_='price regular')
product_list[0].get_text()

#product_price
product_price = [p.get_text() for p in product_list]

In [9]:
# transformando em dataframe
data = pd.DataFrame([product_id, product_cat, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# checando o scrapy datetime
data.head(3)

,product_id,product_category,product_name,product_price,scrapy_datetime
0,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-10-24 20:21:27
1,0985197005,men_jeans_slim,Slim Jeans,$ 19.99,2021-10-24 20:21:27
2,0811993028,men_jeans_regular,Regular Jeans,$ 29.99,2021-10-24 20:21:27
3,0751994034,men_jeans_slim,Slim Jeans,$ 29.99,2021-10-24 20:21:27
4,0938875007,men_jeans_slim,Slim Tapered Jeans,$ 39.99,2021-10-24 20:21:27
5,1004476004,men_jeans_slim,Freefit® Slim Jeans,$ 49.99,2021-10-24 20:21:27
6,0993887002,men_jeans_regular,Hybrid Regular Denim Joggers,$ 39.99,2021-10-24 20:21:27
7,1013317004,men_jeans_regular,Hybrid Regular Tapered Joggers,$ 39.99,2021-10-24 20:21:27
8,0993887001,men_jeans_regular,Hybrid Regular Denim Joggers,$ 39.99,2021-10-24 20:21:27
9,0751994018,men_jeans_slim,Slim Jeans,$ 29.99,2021-10-24 20:21:27


### 1.2.2. Extract II

In [10]:
# checando tota_item
total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')
total_item

'85'

In [11]:
# checando page_number
page_number = int(total_item)/36
page_number

2.361111111111111

In [12]:
# checando page-size
url02 = url + '?page-size' + str(int(page_number*36))
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size85'

### 1.2.3. Extract III (One Product)

In [13]:
#defining the url
url = 'https://www2.hm.com/en_us/productpage.0690449022.html'

#defining the header
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

# ===================================== color name ===========================================

#color name
product_list = soup.find_all('a', class_='filter-option miniature')
color_name = [p.get('data-color') for p in product_list]

# product id
product_id = [p.get('data-articlecode') for p in product_list]

df_color = pd.DataFrame([product_id, color_name]).T
df_color.columns = ['product_id', 'color_name']

# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

# ====================================== composition ==========================================

product_composition_list = soup.find_all('div', class_='pdp-description-list-item')
product_composition = [list( filter(None, p.get_text().split('\n'))) for p in product_composition_list]

# rename dataframe
df_aux = pd.DataFrame(product_composition).T
df_aux.columns = df_aux.iloc[0]

# delete first row
df_aux = df_aux.iloc[1:].fillna(method='ffill')

df_composition = df_aux.copy()

# generate style id + color id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

# merge data color + decomposition
data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size']], how='left', on='style_id')

# checando dados
data_sku.head()

,product_id,color_name,style_id,color_id,Fit,Composition,Size
0,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32"
1,0690449001,Light denim blue/trashed,0690449,001,Skinny fit,"Cotton 98%, Spandex 2%","The model is 184cm/6'0"" and wears a size 31/32"
2,0690449002,Denim blue,0690449,002,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32"
3,0690449002,Denim blue,0690449,002,Skinny fit,"Cotton 98%, Spandex 2%","The model is 184cm/6'0"" and wears a size 31/32"
4,0690449006,Black/washed,0690449,006,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32"
5,0690449006,Black/washed,0690449,006,Skinny fit,"Cotton 98%, Spandex 2%","The model is 184cm/6'0"" and wears a size 31/32"
6,0690449007,Light denim blue,0690449,007,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32"
7,0690449007,Light denim blue,0690449,007,Skinny fit,"Cotton 98%, Spandex 2%","The model is 184cm/6'0"" and wears a size 31/32"
8,0690449009,Black washed out,0690449,009,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32"
9,0690449009,Black washed out,0690449,009,Skinny fit,"Cotton 98%, Spandex 2%","The model is 184cm/6'0"" and wears a size 31/32"


### 1.2.4. Extract IV (Multiple Products)

In [15]:
# definindo headers
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# empty dataframe
df_details = pd.DataFrame()

# unique columns for all products
aux = []

# definindo colunas
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size', 'color_id', 'style_id']
df_pattern = pd.DataFrame(columns=cols)

# criando loop for
for i in range(len(data)):

    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    #defining the header
    
    page = requests.get(url, headers=headers)

    soup = BeautifulSoup(page.text, 'html.parser')

    # ===================================== color name ===========================================

    #color name
    product_list = soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get('data-color') for p in product_list]

    # product id
    product_id = [p.get('data-articlecode') for p in product_list]

    df_color = pd.DataFrame([product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']

    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

    # ====================================== composition ==========================================

    product_composition_list = soup.find_all('div', class_='pdp-description-list-item')
    product_composition = [list( filter(None, p.get_text().split('\n'))) for p in product_composition_list]

    # rename dataframe
    df_aux = pd.DataFrame(product_composition).T
    df_aux.columns = df_aux.iloc[0]

    # delete first row
    df_aux = df_aux.iloc[1:].fillna(method='ffill')
    df_composition = df_aux.copy()
    
    # guarantee the same number of columns
    df_composition = pd.concat([df_pattern, df_composition], axis=0)

    # generate style id + color id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
    
    aux = aux + df_composition.columns.tolist()

    # merge data color + decomposition (size desconsidered because it's not in all products)
    data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size', 'Product safety']], how='left', on='style_id')
    
    # all details products
    df_details = pd.concat([df_details, data_sku], axis=0)

In [16]:
# separando data_style e color_id
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data['color_id'] = data['product_id'].apply(lambda x: x[-3:])

# fazendo o merge nos dados
data_raw = pd.merge(data, df_details[['style_id', 'color_name', 'Fit', 'Composition', 'Size', 'Product safety']], 
         how='left', on='style_id')

In [17]:
# salvando os dados em CSV
data_raw.to_csv('df_jeans.csv', index=False)

# 2.0. Step 02 - Data Cleaning

In [19]:
data = pd.read_csv('df_jeans.csv')
data = data.rename(columns = {'Fit':'fit', 'Composition':'composition', 'Size':'size', 'Product safety': 
                           'product_safety'}, inplace=False)

#product id
data['product_id'] = data['product_id'].astype(int)

# product name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$ ', '')).astype(float)

# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# style id
data['style_id'] = data['style_id'].astype(int)

# color id
data['color_id'] = data['color_id'].astype(int)

# color name
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower())

# fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower())

# size numer
data['size_number'] = data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
data['size_number'] = data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# size model
data['size_model'] = data['size'].str.extract('(\d+/\\d+)')

# composition
data = data[~data['composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['composition'].str.contains('Lining:', na=False)]
data = data[~data['composition'].str.contains('Shell:', na=False)]

# drop duplicates
data = data.drop_duplicates(subset=['product_id', 'product_category', 'product_name', 'product_price',
                                   'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep='last')

# reset index
data = data.reset_index(drop=True)

# break composition by comma
df1 = data['composition'].str.split(',', expand=True)

# cotton / polyester / elastane / elasterell
df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['cotton', 'polyester', 'elastane', 'elasterell'])

# cotton
df_cotton = df1[0].copy()
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

# elastane
df_elastane = df1.loc[df1[1].str.contains('Elastane', na=True), 1]
df_elastane.name = 'elastane'

# combine elastane from both columns 1 and 2
df_elastane = df_elastane.combine_first(df1[2])

df_ref = pd.concat([df_ref, df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elastane'] = df_ref['elastane'].fillna('Elastane 0%')

#elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na=True), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elasterell'] = df_ref['elasterell'].fillna('Elasterell-P 0%')

#final join
data = pd.concat([data, df_ref], axis=1)

#format composition data
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['elastane'] = data['elastane'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['elasterell'] = data['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#dropping columns
data = data.drop(columns=['product_safety', 'size', 'composition'], axis=1)

#drop duplicates
data = data.drop_duplicates()

In [20]:
data.to_csv('jeans_cleaned.csv', index=False)

# 3.0. Step 03 - Database

In [39]:
# carregando os dados
data = pd.read_csv('jeans_cleaned.csv').drop(columns=['color_id'], axis=1)

In [40]:
# preenchedo linhas NA
data['product_category'] = data['product_category'].fillna('jeans')

In [41]:
# criando tabela SQL
query_showroom_schema = """
    CREATE TABLE vitrine(
        product_id          INTEGER, 
        product_category    TEXT, 
        product_name        TEXT, 
        product_price       REAL,
        scrapy_datetime     TEXT, 
        style_id            INTEGER,  
        color_name          TEXT, 
        fit                 TEXT,
        size_number         REAL, 
        size_model          TEXT, 
        cotton              REAL, 
        polyester           REAL, 
        elastane            REAL,
        elasterell          REAL   
    )
"""

In [35]:
# connect to database
conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()
conn.close()

In [42]:
# conectando com o sqlalchemy
conn = create_engine('sqlite:///hm_db.sqlite', echo=False)

In [43]:
# insert data to table
data.to_sql('vitrine', con=conn, if_exists='append', index=False)

In [44]:
# definindo query
query = """
    SELECT * FROM vitrine
"""

# passando para pandas
df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_name,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,690449051,men_jeans_ripped,skinny_jeans,39.99,2021-10-24 20:21:27,690449,black,skinny_fit,187.0,32/32,0.98,0.0,0.0,0.0
1,690449051,men_jeans_ripped,skinny_jeans,39.99,2021-10-24 20:21:27,690449,denim_blue,skinny_fit,NaN,None,0.98,0.0,0.0,0.0
2,690449051,men_jeans_ripped,skinny_jeans,39.99,2021-10-24 20:21:27,690449,light_denim_blue,skinny_fit,NaN,None,0.98,0.0,0.0,0.0
3,690449051,men_jeans_ripped,skinny_jeans,39.99,2021-10-24 20:21:27,690449,black_washed_out,skinny_fit,NaN,None,0.98,0.0,0.0,0.0
4,690449051,men_jeans_ripped,skinny_jeans,39.99,2021-10-24 20:21:27,690449,white,skinny_fit,NaN,None,0.98,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,811993003,men_jeans_regular,regular_jeans,29.99,2021-10-24 20:21:27,811993,denim_blue,regular_fit,NaN,None,0.98,0.0,0.0,0.0
170,811993003,men_jeans_regular,regular_jeans,29.99,2021-10-24 20:21:27,811993,dark_denim_blue,regular_fit,NaN,None,0.98,0.0,0.0,0.0
171,811993003,men_jeans_regular,regular_jeans,29.99,2021-10-24 20:21:27,811993,pale_denim_blue,regular_fit,NaN,None,0.98,0.0,0.0,0.0
172,811993003,men_jeans_regular,regular_jeans,29.99,2021-10-24 20:21:27,811993,blue,regular_fit,NaN,None,0.98,0.0,0.0,0.0
